In [ ]:
!pip install --upgrade openai
import os
import json
import time
from openai import OpenAI 
client = OpenAI(
    api_key=os.getenv("openaikey"))

In [ ]:
user_input = f'I am excited at home thinking about the way to use Functions to generate new types of fully audio controlled app.'
filename = f'memory.json'
function_name = f'add_memory'
function_description =f'Function retrieves user input and saves file using the add_memory function.'
assistant_instruction = r'Convert user (Teemu) inputs to json format with function called {}. Fill all variables and use code interpreter to define exact time at the moment by calling the time package and retrieve current date and time to populate timestamp.  Wait for the function execution and inform user about the retrieved message from the functon executed.'.format(function_name, filename)
model_name = f'gpt-4-1106-preview'
retrieval_tool = {"type": "retrieval"}
code_interpreter_tool = {"type": "code_interpreter"} 
function_tool = {
      "type": "function",
    "function": {
      "name": function_name,
      "description": function_description,
      "parameters": {
        "type": "object",
        "properties": {
          "time_stamp": {"type": "string", "description": "The exact time of the user input, for example 14.11.2023 - 18.45"},
          "subject": {"type": "string", "description": "Describe the subject with 1-2 words, for example Dentist appointment"},
          "description": {"type": "string", "description": "Short summary describing relevant information, for example: Tony reserved dentist. There were no free appointments until end of November 2023."},
          "people": {"type": "string", "description": "Add optionally names or characters of people separated with a comma. For example: Tony, Dentist"},
          "feelings": {"type": "string", "description": "Express optionally emotions separated with a comma, for example: frustrated"},
        },
        "required": ["timestamp", "subject", "description", "people", "feelings"]
      }
    }
  }
assistant_tools = [retrieval_tool, code_interpreter_tool, function_tool]

In [ ]:
def add_memory(timestamp, subject, description, people=None, feelings=None):
    memory_entry = {
        'timestamp': timestamp,
        'subject': subject,
        'description': description,
        'people': people if people else "Not available",
        'feelings': feelings if feelings else "Not available"
        }

    try:
        with open(filename, 'r') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        data = {}

    order_number = len(data) + 1

    data[str(order_number)] = memory_entry

    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=10)
        function_output = r"The new memory records were inserted succesfully."
    except Exception as e:
        function_output = r"The new memory record generation failed."        

    return function_output


In [ ]:
assistant = client.beta.assistants.create(
  instructions= assistant_instruction,
  model=model_name,
  tools= assistant_tools
)

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_input)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)
time.sleep(9.5)
print(run.status)

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id)
time.sleep(2.5)
print(run.status)

In [ ]:
tool_call_id = run.required_action.submit_tool_outputs.tool_calls[0]
function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
function_arguments = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)
print(function_name + "\n" + function_arguments["time_stamp"] + "\n" +function_arguments["subject"]  + "\n" +function_arguments["description"] + "\n" +function_arguments["people"] + "\n" + function_arguments["feelings"])


In [ ]:
function_response = globals()[function_name](function_arguments["time_stamp"], function_arguments["subject"], function_arguments["description"], function_arguments["people"], function_arguments["feelings"])
print(function_response)

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call_id.id,
            "output": function_response,
        }
    ],
)
print(run.status)

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id)
time.sleep(1)
print(run.status)

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id)
print(messages.data[0].content[0].text.value)

In [ ]:
client.beta.assistants.delete(assistant.id)